In [3]:
# Import libraries

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [4]:
# The code was removed by Watson Studio for sharing.

In [5]:
# Filtering the dataframe for One City "Chennai" for the analysis
df_data=df_data_1[df_data_1.Province == "Chennai"]
df_data.head()
print("Locality",len(df_data["Locality"].unique()))
print("Community",len(df_data["Community"].unique()))

Locality 113
Community 15


Chennai City has 15 Communities & 113 Localities

In [6]:
# Identifying the Lat Long of Chennai and mapping it
address = 'Chennai'

geolocator = Nominatim(user_agent="chennai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chennai City are 13.0801721, 80.2838331.


In [9]:
    # create map of Chennai using latitude and longitude values
map_chennai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_data['Lat'], df_data['Long'], df_data['Locality']):
     folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chennai)
   
map_chennai

In [10]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: 2ZIODRA3GZI2ONSVCFFLHBJF1U1CKXGWYFXMZD0LVNPZNQ23
CLIENT_SECRET:QVEDOTROBBVCR4VQTWMAXKEFBVUVRDVDKRWBJ1XVI3U1F154


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Locality', 
                  'Locality Latitude', 
                  'Locality Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
import requests
chennai_venues = getNearbyVenues(names=df_data['Locality'],
                                   latitudes=df_data['Lat'],
                                   longitudes=df_data['Long']
                                  )
print(chennai_venues.shape)
print(chennai_venues.groupby('Locality').count())
chennai_venues.head()

Chennai G.P.O.
MPT AO
Flower Bazaar
Sowcarpet
Mannady (Chennai)
Govt Stanley Hospital
Mint Building
Anna Road
Chintadripet
Madras Electricity System
Edapalayam
Park Town
Ripon Buildings
Madras Medical College
Vivekananda College Madras
Mylapore
Mandaveli
Madras University
Tiruvallikkeni
Chepauk
Teynampet West
Greams Road
Old College Buildings
Shastri Bhavan
Vepery
Egmore
Egmore ND
Ethiraj Salai
Fort St George
Kilpauk
Medavakkam Tank Road
Kilpauk Medical College
Perambur
Perambur North
Sembiam
Perambur Barracks
Venkatesapuram
Puliyanthope
Rayapuram
Triplicane South
Royapettah
Pudupakkam
Lloyds Estate
Saidapet (Chennai)
Guindy North
Thygarayanagar North ND
Hindi Prachar Sabha
Thygarayanagar
Thygarayanagar South NDS.O
Teynampet
Pr. Accountant General
Kasturibai Nagar
Adyar (Chennai)
Shastri Nagar (Chennai)
Washermanpet East
Washermanpet
Rajbhavan (Chennai)
Aynavaram
Kodambakkam
Engineering College (Chennai)
Vadapalani
Raja Annamalaipuram
Ramakrishna Nagar (Chennai)
Aminjikarai
Shenoy Naga

,Locality,Locality Latitude,Locality Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Chennai G.P.O.,13.0933,80.2842,Gujarati Mandal,13.089637,80.285770,Indian Restaurant
1,Chennai G.P.O.,13.0933,80.2842,Hotel Krishna Prasad,13.094429,80.280219,Vegetarian / Vegan Restaurant
2,MPT AO,13.0933,80.2842,Gujarati Mandal,13.089637,80.285770,Indian Restaurant
3,MPT AO,13.0933,80.2842,Hotel Krishna Prasad,13.094429,80.280219,Vegetarian / Vegan Restaurant
4,Flower Bazaar,13.0933,80.2842,Gujarati Mandal,13.089637,80.285770,Indian Restaurant


In [14]:
# one hot encoding
chennai_onehot = pd.get_dummies(chennai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chennai_onehot['Locality'] = chennai_venues['Locality'] 

# move neighborhood column to the first column
fixed_columns = [chennai_onehot.columns[-1]] + list(chennai_onehot.columns[:-1])
chennai_onehot = chennai_onehot[fixed_columns]

chennai_onehot.head()

,Locality,ATM,Accessories Store,African Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Sports Bar,Stadium,Tea Room,Thai Restaurant,Theater,Train,Train Station,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,Chennai G.P.O.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Chennai G.P.O.,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,MPT AO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MPT AO,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Flower Bazaar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
chennai_grouped = chennai_onehot.groupby('Locality').mean().reset_index()
chennai_grouped

,Locality,ATM,Accessories Store,African Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Sports Bar,Stadium,Tea Room,Thai Restaurant,Theater,Train,Train Station,Vegetarian / Vegan Restaurant,Women's Store,Yoga Studio
0,Adyar (Chennai),0.0,0.0,0.000000,0.0,0.000000,0.058824,0.0,0.000000,0.058824,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0
1,Agaram,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0
2,Aminjikarai,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0
3,Anna Nagar East,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.285714,0.000000,0.0
4,Anna Nagar Western Extn,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.285714,0.000000,0.0
5,Arumbakkam,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.0
6,Ashoknagar (Chennai),0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0
7,Aynavaram,0.0,0.0,0.000000,0.0,0.000000,0.058824,0.0,0.000000,0.058824,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0
8,Chennai G.P.O.,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.500000,0.000000,0.0
9,Chepauk,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.00,0.125000,0.125000,0.000000,0.0


In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Locality']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Locality_venues_sorted = pd.DataFrame(columns=columns)
Locality_venues_sorted['Locality'] = chennai_grouped['Locality']

for ind in np.arange(chennai_grouped.shape[0]):
    Locality_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chennai_grouped.iloc[ind, :], num_top_venues)

Locality_venues_sorted.head()

,Locality,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adyar (Chennai),Indian Restaurant,Japanese Restaurant,Bakery,Sandwich Place,Multicuisine Indian Restaurant
1,Agaram,Indie Movie Theater,Department Store,Clothing Store,Gym,Multiplex
2,Aminjikarai,Fast Food Restaurant,Pizza Place,Bookstore,Shopping Mall,Clothing Store
3,Anna Nagar East,Vegetarian / Vegan Restaurant,Food Court,Hotel,Fast Food Restaurant,Bus Station
4,Anna Nagar Western Extn,Vegetarian / Vegan Restaurant,Food Court,Hotel,Fast Food Restaurant,Bus Station


In [67]:
# set number of clusters
kclusters = 10

chennai_grouped_clustering = chennai_grouped.drop('Locality', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chennai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 0, 0, 0, 4, 4, 8, 2], dtype=int32)

In [68]:
# add clustering labels
Locality_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chennai_merged = df_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
chennai_merged = chennai_merged.join(Locality_venues_sorted.set_index('Locality'), on='Locality')




In [79]:
#chennai_merged.dropna()
chennai_merged['Cluster Labels'].unique()
chennai_merged['Cluster Labels']=chennai_merged['Cluster Labels'].fillna(0)

#chennai_merged.head()


array([ 2.,  0., nan,  1.])

In [71]:
chennai_merged['Cluster Labels']=chennai_merged['Cluster Labels'].astype('int')
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chennai_merged['Lat'], chennai_merged['Long'], chennai_merged['Locality'], chennai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [87]:
# Hierarchical Clustering
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=8, affinity='euclidean', linkage='ward')  
hier=cluster.fit_predict(chennai_grouped_clustering)
hier

array([2, 0, 0, 0, 0, 0, 2, 2, 6, 2, 2, 0, 2, 2, 2, 0, 2, 5, 2, 6, 2, 7,
       0, 0, 6, 7, 0, 3, 0, 3, 0, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2,
       0, 6, 2, 2, 2, 2, 2, 2, 6, 2, 2, 0, 0, 7, 2, 2, 3, 2, 2, 2, 3, 0,
       2, 0, 2, 2, 2, 0, 3, 2, 2, 7, 2, 2, 2, 0, 2, 7, 3, 3, 3, 0, 2, 2,
       4, 4, 4, 2, 0, 2, 3, 0, 0, 2, 1, 1, 0, 0, 2, 2])

In [89]:
# add clustering labels
Locality_venues_sorted.insert(0, 'Hier Labels', hier)

chennai_merged = df_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
chennai_merged = chennai_merged.join(Locality_venues_sorted.set_index('Locality'), on='Locality')

In [90]:
chennai_merged['Hier Labels'].unique()
chennai_merged['Hier Labels']=chennai_merged['Hier Labels'].fillna(0)


In [91]:
chennai_merged['Hier Labels']=chennai_merged['Hier Labels'].astype('int')
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chennai_merged['Lat'], chennai_merged['Long'], chennai_merged['Locality'], chennai_merged['Hier Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters